# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code

# Important Note
Used an open-source model gemini-1.5-pro ,can try 2.0 flash too


In [ ]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
import google.generativeai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [ ]:
# environment

load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    print(f"Google API Key exists")
else:
    print("Google API Key not set")

In [ ]:
google.generativeai.configure()


In [ ]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Windows system. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [ ]:
def optimize_google(python):
    # Initialize empty reply string
    reply = ""
    
    # The API for Gemini has a slightly different structure
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-1.5-pro',
        system_instruction=system_message
    )
    
    response = gemini.generate_content(
        user_prompt_for(python),
        stream=True
    )
    
    # Process the stream
    for chunk in response:
        # Extract text from the chunk
        if chunk.text:
            reply += chunk.text
            print(chunk.text, end="", flush=True)
    
    # Write the complete response to output
    write_output(reply)
    
    # return reply

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
exec(pi)

In [ ]:
optimize_google(pi)

In [ ]:
exec(pi) #Execution Time: 16.209231 seconds

# Compiling C++ and executing

This next cell contains the command to compile a C++ file on Windows system.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

The way to compile for mac users is \
!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp \
!./optimized

In [ ]:
!g++ -o optimized optimized.cpp
!.\optimized #Execution Time: 3.661196 seconds

In [ ]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard) #Execution Time: 62.297366 seconds

In [ ]:
optimize_google(python_hard)

In [ ]:
!g++ -o optimized optimized.cpp
!.\optimized

In [ ]:
def stream_google(python):
    # Initialize empty reply string
    reply = ""
    
    # The API for Gemini has a slightly different structure
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-1.5-pro',
        system_instruction=system_message
    )
    
    response = gemini.generate_content(
        user_prompt_for(python),
        stream=True
    )
    
    # Process the stream
    for chunk in response:
        # Extract text from the chunk
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```cpp\n','').replace('```','')


In [ ]:
def optimize(python, model):
    result=stream_google(python)
    for stream_so_far in result:
        yield stream_so_far    

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["Google"], label="Select model", value="Google")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [ ]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [ ]:
def execute_cpp(code):
        write_output(code)
        try:
            # compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_cmd = ["g++", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["Google"], label="Select model", value="Google")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)